In [ ]:
STORAGE_PATH = "gs://mlflow-bucket-ehp/algo_training_stg/algo_training_v2_two_tower_20230111T094741"

# Get data

In [ ]:
import pandas as pd

raw_data = pd.read_gbq(
    """
SELECT
    booking.user_id,
    CAST(enruser.user_age AS INT64) AS user_age,
    enruser.user_postal_code,
    enruser.user_activity,
    enruser.booking_cnt AS user_booking_cnt,
    ROUND(enruser.theoretical_amount_spent, -1) AS user_theoretical_amount_spent,
    ROUND(enruser.user_theoretical_remaining_credit, -1) AS user_theoretical_remaining_credit,
    enruser.cnt_distinct_type_booking AS user_distinct_type_booking_cnt,

    offer_item_ids.item_id as item_id,
    subcategories.category_id as offer_categoryId,
    offer.offer_subcategoryId as offer_subcategoryid,
    STRING_AGG(enroffer.offer_name, ' ') AS item_names,
    STRING_AGG(enroffer.rayon, ' ') AS item_rayons,
    STRING_AGG(enroffer.author, " ") AS item_author,
    STRING_AGG(enroffer.performer, " ") AS item_performer,
    ROUND(AVG(enroffer.last_stock_price), -1) AS item_mean_stock_price,
    ROUND(SUM(enroffer.booking_confirm_cnt), -1) AS item_booking_cnt,
    ROUND(SUM(enroffer.favourite_cnt), -1) AS item_favourite_cnt,

    COUNT(*) as count
from
    `passculture-data-ehp.clean_stg`.`applicative_database_booking` booking
    inner join `passculture-data-ehp.clean_stg`.`applicative_database_stock` stock on booking.stock_id = stock.stock_id
    inner join `passculture-data-ehp.clean_stg`.`applicative_database_offer` offer on stock.offer_id = offer.offer_id
    inner join `passculture-data-ehp.analytics_stg`.`subcategories` subcategories on offer.offer_subcategoryId = subcategories.id
    inner join `passculture-data-ehp.analytics_stg`.`enriched_offer_data` enroffer on enroffer.offer_id = offer.offer_id
    inner join `passculture-data-ehp.analytics_stg`.`offer_item_ids` offer_item_ids on offer_item_ids.offer_id = offer.offer_id
    inner join `passculture-data-ehp.analytics_stg`.`enriched_user_data` enruser on enruser.user_id = booking.user_id
where
    booking.booking_creation_date >= DATE_SUB(DATE("2023-01-12"), INTERVAL 8 MONTH)
    and booking.booking_creation_date <= DATE("2023-01-12")
    and booking.user_id is not null
group by
    booking.user_id,
    enruser.user_age,
    enruser.user_postal_code,
    enruser.user_activity,
    enruser.booking_cnt,
    enruser.theoretical_amount_spent,
    enruser.user_theoretical_remaining_credit,
    enruser.cnt_distinct_type_booking,

    offer_item_ids.item_id,
    subcategories.category_id,
    offer.offer_subcategoryId
    """
)

In [ ]:
raw_data.describe()

In [ ]:
raw_data.to_csv(f"{STORAGE_PATH}/raw_data.csv", index=False)

# Preprocess

In [ ]:
clean_data = raw_data.fillna(
    {
        "user_postal_code": "none",
        "user_activity": "none",
        "user_booking_cnt": 0,
        "user_theoretical_amount_spent": 0,
        "user_theoretical_remaining_credit": 0,
        "user_distinct_type_booking_cnt": 0,
        "item_names": "none",
        "item_rayons": "none",
        "item_author": "none",
        "item_performer": "none",
        "item_mean_stock_price": 0,
        "item_booking_cnt": 0,
        "item_favourite_cnt": 0,
    }
).astype(
    {
        "user_theoretical_amount_spent": "int",
        "user_theoretical_remaining_credit": "int",
        "user_distinct_type_booking_cnt": "int",
        "item_mean_stock_price": "int",
        "item_booking_cnt": "int",
        "item_favourite_cnt": "int",
    }
)

In [ ]:
clean_data.describe()

In [ ]:
clean_data.to_csv(f"{STORAGE_PATH}/clean_data.csv", index=False)

# Split data

In [ ]:
from split_data import main

main(STORAGE_PATH)

In [ ]:
pd.read_csv(f"{STORAGE_PATH}/positive_data_eval.csv").describe()
